# Chain 拆解實作範例 - 簡化版

本範例展示如何將 LCEL Chain 拆解成 RunnableLambda，幫助理解 Chain 的內部運作機制。

## 學習目標
- 理解 Chain 的組成元件
- 學會將處理步驟拆解成獨立函數
- 掌握 RunnableLambda 的使用方法
- 比較 LCEL 語法與手動組合的差異

## 步驟 1: 導入套件並設定模型

In [4]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain_ollama.llms import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立模型
model = OllamaLLM(model="llama3.2:latest")

## 步驟 2: 建立提示模板

In [5]:
# 建立一個簡單的問答系統
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個友善的助手，請用繁體中文回答問題。"),
    ("human", "{question}")
])

print("提示模板建立完成！")

提示模板建立完成！


## 步驟 3: 使用 LCEL 語法（傳統方式）

先看看我們習慣的寫法：

In [6]:
# 使用 LCEL 語法建立 Chain
simple_chain = prompt_template | model

# 測試執行
result = simple_chain.invoke({"question": "什麼是 Python?"})
print("LCEL 回答：")
print(result)

LCEL 回答：
Python 是一個強大的、易學的Programming語言，常被用於開發不同領域的應用程式。它是一種高級語言，主要用於創建大型計算機程式、網路應用程式和Scientific Computing等方面。 Python 的優點包括： 

*   超快的程序執行速度
*   簡單易學的程序風格
*   允許多第三方庫和框架的使用
*   跨platform compatibility（可在不同operating system上運行）


## 步驟 4: 拆解成獨立函數

現在我們來理解 Chain 內部做了什麼事：

In [7]:
# 拆解步驟 1: 格式化提示
def step1_format_prompt(input_dict):
    """將輸入字典轉換成格式化的提示"""
    print(f"\n[步驟1] 輸入: {input_dict}")
    formatted = prompt_template.format_prompt(**input_dict)
    print(f"[步驟1] 輸出類型: {type(formatted).__name__}")
    return formatted

# 拆解步驟 2: 呼叫模型
def step2_call_model(formatted_prompt):
    """將格式化的提示傳給模型"""
    print(f"\n[步驟2] 輸入類型: {type(formatted_prompt).__name__}")
    messages = formatted_prompt.to_messages()
    response = model.invoke(messages)
    print(f"[步驟2] 輸出類型: {type(response).__name__}")
    return response

# 拆解步驟 3: 回傳結果
def step3_return_result(response):
    """回傳最終結果"""
    print(f"\n[步驟3] 輸入類型: {type(response).__name__}")
    print(f"[步驟3] 最終輸出: {response[:50]}..." if len(response) > 50 else f"[步驟3] 最終輸出: {response}")
    return response

print("拆解函數定義完成！")

拆解函數定義完成！


## 步驟 5: 用 RunnableLambda 包裝函數

In [8]:
# 將每個函數包裝成 RunnableLambda
runnable_step1 = RunnableLambda(step1_format_prompt)
runnable_step2 = RunnableLambda(step2_call_model)
runnable_step3 = RunnableLambda(step3_return_result)

print("RunnableLambda 包裝完成！")
print(f"步驟1: {runnable_step1}")
print(f"步驟2: {runnable_step2}")
print(f"步驟3: {runnable_step3}")

RunnableLambda 包裝完成！
步驟1: RunnableLambda(step1_format_prompt)
步驟2: RunnableLambda(step2_call_model)
步驟3: RunnableLambda(step3_return_result)


## 步驟 6: 用 RunnableSequence 組合

In [9]:
# 手動組合 Chain
manual_chain = RunnableSequence(
    first=runnable_step1,
    middle=[runnable_step2],
    last=runnable_step3
)

print("手動組合的 Chain 建立完成！")

手動組合的 Chain 建立完成！


## 步驟 7: 執行並觀察資料流動

In [10]:
# 執行手動組合的 Chain
print("="*60)
print("開始執行手動組合的 Chain")
print("="*60)

result = manual_chain.invoke({"question": "什麼是 LangChain?"})

print("\n" + "="*60)
print("最終結果：")
print("="*60)
print(result)

開始執行手動組合的 Chain

[步驟1] 輸入: {'question': '什麼是 LangChain?'}
[步驟1] 輸出類型: ChatPromptValue

[步驟2] 輸入類型: ChatPromptValue
[步驟2] 輸出類型: str

[步驟3] 輸入類型: str
[步驟3] 最終輸出: LangChain 是一個開源的 Python 库，旨在簡化自然語言處理的作業。它提供了一種方便和有...

最終結果：
LangChain 是一個開源的 Python 库，旨在簡化自然語言處理的作業。它提供了一種方便和有效的方式，用於建立、管理和調整不同型別的 language model 和其相關元資料。

LangChain 會用來實現多種用途，例如：

*   認識與生成文本
*   文本類別化
*   自動對語言數據進行分析和整理
*   等等

 LangChain 是一種開源的 Python 库，所以使用它的人可以透過 open source 的方式來獲得更多的功能和 customization 的選項。


## 步驟 8: 更簡潔的寫法（使用 Lambda 表達式）

In [11]:
# 不需要先定義函數，直接用 lambda 表達式
compact_chain = RunnableSequence(
    first=RunnableLambda(lambda x: prompt_template.format_prompt(**x)),
    middle=[RunnableLambda(lambda x: model.invoke(x.to_messages()))],
    last=RunnableLambda(lambda x: x)
)

# 測試執行
result = compact_chain.invoke({"question": "LangChain 有什麼優點?"})
print("簡潔版回答：")
print(result)

簡潔版回答：
LangChain 是一個開源的程式框架，可以幫助創建和使用人工智慧（AI）模型。以下是它的一些優點：

1. **易於使用**：LangChain 提供了簡單且清晰的 API，讓使用者可以輕鬆地整合 AI 模型到自己的程式中。
2. **多樣性**：LangChain 支持多種 AI_frameworks，如 Hugging Face、PyTorch 等，可以與不同的模型一起使用。
3. **可擴展性**：LangChain 的 API 可以被 custom化和擴展，讓使用者可以添加新的功能或增強現有的功能。
4. **安全性**： LangChain 適合使用 HTTPS 協議傳遞數據，以確保資料的安全性和隱私權。
5. **開源**： LangChain 是一個免費的開源程式框架，讓所有人都可以使用和贈與。
6. **快速開發**： LangChain 可以幫助減少 AI 相關開發的時間，由於它提供了一個簡單和有效的 API。

總之，LangChain 是一個優雅、易用的程式框架，可以幫助創建和使用人工智慧模型。


## 總結：三種寫法比較

### 1️⃣ LCEL 語法（最簡潔）
```python
chain = prompt_template | model
```
- ✅ 程式碼最短
- ✅ 適合生產環境
- ❌ 無法看到內部運作

### 2️⃣ 手動拆解（最清楚）
```python
def step1_format_prompt(x): ...
def step2_call_model(x): ...
chain = RunnableSequence(
    first=RunnableLambda(step1_format_prompt),
    middle=[RunnableLambda(step2_call_model)],
    last=...
)
```
- ✅ 完全掌控每個步驟
- ✅ 適合除錯和學習
- ✅ 可以加入自訂邏輯
- ❌ 程式碼較長

### 3️⃣ Lambda 表達式（折衷）
```python
chain = RunnableSequence(
    first=RunnableLambda(lambda x: ...),
    middle=[RunnableLambda(lambda x: ...)],
    last=...
)
```
- ✅ 比手動拆解簡潔
- ✅ 仍有彈性可調整
- ❌ 複雜邏輯不易閱讀

## 🎯 教學建議
1. **初學者**：從手動拆解開始，理解每個步驟
2. **進階者**：使用 Lambda 表達式，快速調整
3. **專案開發**：使用 LCEL 語法，保持程式碼簡潔

## 💡 何時需要 RunnableLambda？
- 需要**加入日誌記錄**
- 需要**資料預處理**（如格式轉換、驗證）
- 需要**條件判斷**（根據結果決定下一步）
- 需要**呼叫外部 API** 或資料庫
- 需要**錯誤處理**和例外捕捉